# SEGMENT DATA FROM NEARBY NODES
In this notebook, we make a subselection of the sessionGraph and segment the data
As ouput, the method generates a selection of data

>This codebase operates on the scan2bim2.yml environment (python 3.8)

In [3]:
#IMPORT PACKAGES
import rdflib
from rdflib import Graph, plugin
from rdflib.serializer import Serializer #pip install rdflib-jsonld https://pypi.org/project/rdflib-jsonld/
from rdflib import Graph
from rdflib import URIRef, BNode, Literal
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD
import trimesh 

import os.path, time
import importlib
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
import ifcopenshell.util.selector
import operator
import copy

#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
%load_ext autoreload

In [5]:
%autoreload 2

## 1. INPUTS

In [6]:
## INPUTS
projectPath= os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)),"tests")
sessionPath = os.path.join(projectPath, "Samples3" )

bimGraphPath=os.path.join(sessionPath,'bimGraph.ttl')
meshPath=os.path.join(sessionPath,'week22.obj')

u=2.0 # (float) search distance in X given the boundingbox
v=2.0 # (float) search distance in Y given the boundingbox
z=1.0 # (float) search distance in Z given the boundingbox

## 2. READ Data

In [7]:
# Get BIM Nodes & calculate their Bboxes
graph=Graph()
graph.parse(bimGraphPath) 
BIMNodelist=tl.graph_to_nodes(graph,sessionPath=sessionPath)
    
# get/set Mesh Node
meshNode=MeshNode(path=meshPath)

In [8]:
mesh=meshNode.mesh
triangleIndices=[0,1,2]
# triangleIndices=range(0,len(mesh.triangles))
print(len(triangleIndices))



3


In [9]:
triangles=np.asarray([triangle for idx,triangle in enumerate(mesh.triangles) if idx in triangleIndices])
print(triangles)

[[0 1 2]
 [3 4 5]
 [6 7 8]]


In [11]:
import time
st = time.time()

centers=gmu.get_triangles_center(mesh,triangleIndices)
print(centers)
et = time.time()
print("test_get_center. time: "+str(et - st))

[[ 4.48127794 57.81929525 15.08696365]
 [-4.04033788 75.143514    4.46169472]
 [-4.02343432 75.10202026  5.12661028]]
test_get_center. time: 0.9296307563781738


# 5. SEGMENTATION 1: oriented Bounding Box
select mesh triangles that lie within the oriented bounding box of a mesh

In [ ]:
# get all boxes
for node in BIMNodelist:
    if getattr(node,'orientedBounds',None) is not None:
        node.box=tl.oriented_bounds_to_open3d_oriented_bounding_box(node.orientedBounds)   
        node.box=node.box.translate([0,0,-4])
        node.box=gmu.expand_box(node.box,u=u,v=v,z=z)
        node.box.color=[0,1,0]   

In [ ]:
#show boxes
# idx=2
# BIMNodelist[idx].box=gmu.expand_box(BIMNodelist[idx].box,u=u,v=v,z=z)
# BIMNodelist[idx].box.color=[1,0,0]
boxes=[node.box for node in BIMNodelist]
o3d.visualization.draw_geometries(boxes+[meshnode.mesh])

In [14]:
print('{:50s} {:5s} '.format('tests','time'))


tests                                              time  


In [ ]:
#crop boxes
croppedMesh=[None] * len(BIMNodelist)
for node in BIMNodelist:
    color=np.random.rand(3,1)
    node.box.color=color
    node.croppedMesh=gmu.crop_geometry(meshNnode.mesh, node.box,subdivide=0)
    if node.croppedMesh != None:
        node.croppedMesh.paint_uniform_color(color)

In [ ]:
# visMeshes=[]
# for mesh in croppedMesh:
#     if mesh is not None:
#         visMeshes.append(mesh.paint_uniform_color (np.random.rand(3,1)))
boxes=[node.box for node in BIMNodelist if node.box != None]
croppedMeshes=[node.croppedMesh for node in BIMNodelist if node.croppedMesh != None]
o3d.visualization.draw_geometries(croppedMeshes + boxes)

# 6. SEGMENTATION2: trimesh convex hull intersection
cut a mesh that lies within the convex hull of another mesh

In [ ]:
#create trimesh geometries & compute intersections
triMesh= gmu.open3d_to_trimesh(meshNnode.mesh)
for node in BIMNodelist: 
    triBox= gmu.open3d_to_trimesh(node.box)
    new_mesh=gmu.mesh_intersection_convex_hull(triMesh,triBox,inside=True)
    node.trimesh=new_mesh.as_open3d
    node.trimesh.paint_uniform_color(node.box.color)

In [ ]:
boxes=[node.box for node in BIMNodelist if node.box != None]
meshes=[node.trimesh for node in BIMNodelist if node.trimesh != None]
o3d.visualization.draw_geometries(meshes + boxes)

# 7. SEGMENTATION3: Open3D raycast points
determine which points of a mesh lie within a closed mesh (accurate geometry)

In [ ]:
for node in BIMNodelist: 
    shape=gmu.open3d_box_to_mesh(node.box)
    node.raymesh=gmu.mesh_intersection_raycasting(meshNnode.mesh, shape, inside = True,strict = True) # this is getting fairly slow
    node.raymesh.paint_uniform_color(node.box.color)

In [ ]:
boxes=[node.box for node in BIMNodelist if node.box != None]
meshes=[node.raymesh for node in BIMNodelist if node.raymesh != None]
o3d.visualization.draw_geometries(meshes + boxes)